This notebook is a experiment of creating model, especially Feed-Forward Neural Network for predicting the closing price of EURUSD based on daily closing price


In [0]:
#Necessary Imports
import pandas as pd
import numpy as np

printa

